Get IES coordinates from MAC+IES assembly and convert them to coordinates relative to MAC reference assembly.

Compare against the BleTIES predictions.

In [1]:
from Bio import SeqIO
from Bio import Seq
import re
from collections import defaultdict
import json

In [2]:
ls ref

ptetraurelia_mac_51.fa
ptetraurelia_mac_51.min100k_max100N.fa
ptetraurelia_mac_51.min100k_max100N.fa.fai
ptetraurelia_mac_51.min100k_max100N.iescoords.json
ptetraurelia_mac_51_with_ies.fa
ptetraurelia_mac_51_with_ies.min100k_max100N.fa


In [3]:
%%bash
grep -c '>' ref/ptetraurelia_mac_51_with_ies.min100k_max100N.fa

61


In [4]:
macies = SeqIO.to_dict(SeqIO.parse("ref/ptetraurelia_mac_51_with_ies.min100k_max100N.fa", "fasta"))

In [5]:
mac = SeqIO.to_dict(SeqIO.parse("ref/ptetraurelia_mac_51.min100k_max100N.fa", "fasta"))

In [6]:
print("MAC+IES")
print(str(macies['scaffold51_147_with_IES'].seq[1751:1791]))
print(str(macies['scaffold51_147_with_IES'].seq[1781:1821]))
print("MAC")
print(str(mac['scaffold51_147'].seq[1751:1791]))
print(str(macies['scaffold51_147_with_IES'].seq[1751:1791]))

MAC+IES
CAATAACATTTATGTTCTCTtattaagaaattaaaataac
ttaaaataacattatgaataATAACAATTTCTTTTAATTT
MAC
CAATAACATTTATGTTCTCTTAATAACAATTTCTTTTAAT
CAATAACATTTATGTTCTCTtattaagaaattaaaataac


The MAC+IES assembly has the entire IES sequence in lower case, including both of the terminal TA-repeats. 

This means that when we subtract IES lengths in getting coordinates of the IESs relative to the MAC reference, we should subtract 2 bp from the IES length.

In [6]:
# Get coordinates of softmasked (lower-cased) regions in the MAC+IES assembly
iescoords = defaultdict(list)
for scaff in macies:
    ff = re.finditer(r'[atcg]+', str(macies[scaff].seq))
    for i in ff:
        iescoords[scaff].append(i.span())

In [8]:
# export as JSON
with open("ref/ptetraurelia_mac_51_with_ies.min100k_max100N.iescoords.json", "w") as fh:
    fh.write(json.dumps(iescoords,indent=4))

In [9]:
# Adjust coordinates if IES segments removed
iescoords_adj = {}
for scaff in iescoords:
    scaff_orig = scaff[:-9] # remove suffix _with_IES from scaff name
    runningtotal = 0
    adjcoords = []
    ieslens = []
    for i in range(len(iescoords[scaff])):
        ieslen = iescoords[scaff][i][1] - iescoords[scaff][i][0] - 2
        adjcoords.append(iescoords[scaff][i][0] - runningtotal)
        ieslens.append(ieslen)
        runningtotal += ieslen
        # subtract 2 because softmasked region includes both TA flanking repeats
        # but one of them is retained in the MAC MDS
    iescoords_adj[scaff_orig] = dict(zip(adjcoords,ieslens))

In [10]:
# Total number of IESs
sum([len(iescoords_adj[scaff]) for scaff in iescoords_adj])

12199

In [12]:
# Number of IESs per scaffold
for scaff in iescoords_adj:
    print(scaff + "\t" + str(len(iescoords_adj[scaff])))

scaffold51_147	192
scaffold51_151	176
scaffold51_103	235
scaffold51_175	62
scaffold51_74	223
scaffold51_131	189
scaffold51_61	244
scaffold51_167	166
scaffold51_139	193
scaffold51_169	104
scaffold51_174	68
scaffold51_140	157
scaffold51_73	219
scaffold51_128	162
scaffold51_171	175
scaffold51_65	288
scaffold51_115	168
scaffold51_41	285
scaffold51_63	244
scaffold51_85	231
scaffold51_66	249
scaffold51_45	274
scaffold51_132	189
scaffold51_125	214
scaffold51_30	269
scaffold51_153	159
scaffold51_11	341
scaffold51_134	177
scaffold51_127	178
scaffold51_14	402
scaffold51_114	217
scaffold51_149	133
scaffold51_129	81
scaffold51_79	215
scaffold51_135	153
scaffold51_33	310
scaffold51_94	181
scaffold51_110	167
scaffold51_104	229
scaffold51_101	189
scaffold51_164	134
scaffold51_39	381
scaffold51_83	258
scaffold51_91	199
scaffold51_116	208
scaffold51_121	180
scaffold51_87	115
scaffold51_108	201
scaffold51_46	325
scaffold51_137	120
scaffold51_148	192
scaffold51_98	192
scaffold51_179	103
scaffold51_152	14

In [15]:
# Sanity check - compare 10 IESs and flanking MDS regions in both assemblies
# to make sure that the coordinates match up
#for i in range(len(iescoords_adj['scaffold51_147_with_IES'])):
for i in range(10):
    coord_adj = list(iescoords_adj['scaffold51_147'].keys())[i]
    coord = iescoords['scaffold51_147_with_IES'][i][0]
    coord1 = iescoords['scaffold51_147_with_IES'][i][1]
    print(coord_adj)
    print(str(macies['scaffold51_147_with_IES'].seq)[coord-20:coord+20])
    print(str(mac['scaffold51_147'].seq)[coord_adj-20:coord_adj+20])
    print(str(macies['scaffold51_147_with_IES'].seq)[coord1-22:coord1+18])
    print(str(macies['scaffold51_147_with_IES'].seq)[coord:coord1])

1771
CAATAACATTTATGTTCTCTtattaagaaattaaaataac
CAATAACATTTATGTTCTCTTAATAACAATTTCTTTTAAT
aattaaaataacattatgaataATAACAATTTCTTTTAAT
tattaagaaattaaaataacattatgaata
2480
TTATATTAATTAATTTTGAGtattctaaaatgcaaacact
TTATATTAATTAATTTTGAGTAATGATTTGTCTATTGAAA
aatgcaaacactcttttcattaATGATTTGTCTATTGAAA
tattctaaaatgcaaacactcttttcatta
2571
TAATATTTTTCCAATTTATAtattgctagatgagaaatta
TAATATTTTTCCAATTTATATAGAGCAAATCCTTCCATTA
atgagaaattaaaattttattaGAGCAAATCCTTCCATTA
tattgctagatgagaaattaaaattttatta
2800
TCAAATTTAATTTGATGCTTtataataaaatatagaataa
TCAAATTTAATTTGATGCTTTAATGGAAGGCCTTTGAAAT
aatatagaataatctatgagtaATGGAAGGCCTTTGAAAT
tataataaaatatagaataatctatgagta
3407
CAGTTGGAATAAGCAGTTATtacacttcaacattttaaag
CAGTTGGAATAAGCAGTTATTATTTTATGCTATTAACTAA
ctttttgagaggatagactgtaTTTTATGCTATTAACTAA
tacacttcaacattttaaaggttattcaaaaatatactttttgagaggatagactgta
5728
AGATCCAATGATAAGGTTGTtatagggatttctgaaacta
AGATCCAATGATAAGGTTGTTATCAATTGGGTTGAATATG
ttctgaaactaaacaaaccgtaTCAATTGGGTTGAATATG
tatagggatttctgaaactaaacaaaccgta
8585
AATTGATTTAT

In [16]:
# Export as JSON
with open("ref/ptetraurelia_mac_51.min100k_max100N.iescoords.json", "w") as fh:
    fh.write(json.dumps(iescoords_adj,indent=4))